 <h2>LSTM Recurrent Neural Network for code generation of opencv library in a many to many fashion using time distributed dense layer.</h2> 
 Recurrent neural networks : From one moment to the next, our brain operates as a function. it accepts inputs from our senses and our thoughts and produces in the form of actions and new thoughts. we see a bear and then think "bear". we can model this behaviour with a feed forward neural networks. we can teach a feed forward neural network to think "bear" when it is shown an image of a bear. 
 But our brain is not a one-shot function. It runs repeatedly through time. we see a "bear" then think "run". Importantly, the very same function that transforms the image of the bear into the thought "bear" into the thought "run". It is a recurring function, which we can model with a recurrent neural network(RNN).
 An RNN is a composition of identical feed forward neural networks, one for each moment, or step in time, which we will refer to as 'RNN cells'. these cells operate on their own output, allowing allowing them to be composed. They can also operate on external input and produce external output. Here is a diagram  of a single RNN cell: 
 <h2></h2> <img src =" https://r2rt.com/static/images/NH_SingleRNNcell.png"> 
 
 The two main problem associated with Recurrent neural network are 1. Information morphing : In ability to keep the memory content for more than a few time steps. 2. Vanishing and Exploding gradients: because, backpropogation through time makes them incredibly deep networks. if the gradients explode, we can't train our model. If they vanish, It's difficult for us to learn long-term dependencies.
 
 To address these issues Sepp Hochreiter and Jurgen Schmidhuber introduced the Long Short Term Memory in the year 1997.

Long Short Term Memory:
The Fundamental principle of LSTM's: To ensure the integrity of our messages in the real world, we write them down. writing is a delta to the current state: it is an act of creation or destruction; the subject itself does not morph when you write on it and other thing is the error gradient on the backward-pass is constant, that can be done by an explicit addition or subtraction, so that each element of the state stays constant without outside interference: "the unit's activation has to remain constant this will be ensured by using the identity function". 
The Fundamental challenge of LSTM's: Uncontrolled and uncordinated writing. particularly at the start of training when writes are completely random, create a chaotic state that leads to bad results and from which it can be difficult to recover. 
To overcome this challenge, the author decided to using selectivity to control and coordinate writing by keeping our state under control is to be selective in three things: what we write, what we read(because we need to read something to know what to write), and what we forget(because obsolete information is a distraction and should be forgotten). The Selective reading, writing and forgetting involves seperate read, write and forget decisions for each element of the state. we will make these decisions by taking advantage of state-sized read, write and forget vectors with value between 0 and 1specifying the percentage of reading, writing and forgetting that we do for each state element. Note that while it may be more natural to think of reading, writing and forgetting as a binary decisions, we need our decisions to be implented via a differentiable function. the logistic sigmoid is a natural choice since it is differentiable and produces continues values between 0 and 1. we call these read, write and forget vectors "gates".  our three gates at time step t is denoted as it, the input gate(for writing), ot, the output gate(for reading) and ft, the forget gate(for remembering). st is our candidate write. and ⊙ denotes elementwise multiplication. xt is the input information and rnn out is the output of the single cell. the below diagram shows the data flow of the LSTM.     <h2></h2> <img src =" https://r2rt.com/static/images/NH_PrototypeLSTMCell.png"> 
<h2></h2> <img src =" https://cdn-images-1.medium.com/max/1600/0*HO2RAef8iFyAJ04V.png"> 


Here, the main goal of the LSTM RNN network is to generate a opencv source code for a given seed string frm the opencv source code. here we used many to many  model using TimeDistributedDense layer. TimeDistributedDense layer is used to keep one to one relations on input and output. Assume you have 60 time steps with 100 samples of data which is (60 X 100). and you want use RNN with output of 200. If you dont use TImeDIstributedDense Layer, You'll get 100 X 60 X 200 tensor. so you have the output flattened with each timestep mixed. If you apply TimeDistributedDense , you are going to apply fully connected dense on each time step and get output seperately by timesteps. In this project we have taken the all source code of the opencv and converted that into long text format and then we used that text file to feed that into network model. steps to convert all the source code to text format: step1. clone the opencv repository "git clone https://github.com/opencv/opencv.git". step 2: change directory to opencv "cd opencv". step 3: run this command by giving destination folder as the argument "find . -name "*.[c|h]" | shuf | xargs cat > opencv.txt". this is the peocedure to convert all the source code to single text file. then we built LSTM model then we feed that data into it trained with iteratively. once the model is build then we took the piece of the corpus from the large chunk of the data and used that as the seed string to generate the source code. finally we gave the length of code to be generate.     




In [0]:
#define the necessary libraries
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM,TimeDistributed,SimpleRNN
from keras.utils.data_utils import get_file
import numpy as np
from time import sleep
import random
import sys


In [0]:
#Loading the complete opencv source code data in the form of text file from the google drive

path = get_file('opencv.txt', origin="https://drive.google.com/drive/u/0/folders/18LEsdM1q3wv3Ri8FsSjYR5xn51nH5aBZ")

text = open(path).read()

# this finds the number of words
print('corpus length:', len(text))

# it sorts the all character in the form of particular order
chars = sorted(list(set(text)))
print('total chars:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# split the corpus into sequences of length=maxlen
#input is a sequence of 40 chars and target is also a sequence of 40 chars shifted by one position
#for eg: if you maxlen=3 and the text corpus is abcdefghi, your input ---> target pairs will be
# [a,b,c] --> [b,c,d], [b,c,d]--->[c,d,e]....and so on
maxlen = 40
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen+1, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i+1:i +1+ maxlen])
    #print('nb sequences:', len(sentences))

print('Vectorization...')
#it will convert each word into a one hot vector
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences),maxlen, len(chars)), dtype=np.bool) # y is also a sequence , or  a seq of 1 hot vectors
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1

for i, sentence in enumerate(next_chars):
    for t, char in enumerate(sentence):
        y[i, t, char_indices[char]] = 1
    

print ('vetorization completed')




corpus length: 71071
total chars: 180
Vectorization...
vetorization completed


In [0]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print ('model is made')

# train the model, output generated text after each iteration

Build model...
model is made


In [0]:
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 40, 512)           1419264   
_________________________________________________________________
lstm_12 (LSTM)               (None, 40, 512)           2099200   
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 40, 180)           92340     
_________________________________________________________________
activation_6 (Activation)    (None, 40, 180)           0         
Total params: 3,610,804
Trainable params: 3,610,804
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
#at each iteration it does 5 epochs  
for iteration in range(1, 6):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    history=model.fit(X, y, batch_size=128, nb_epoch=5,verbose=1)
    sleep(0.1) # python need some time before fitting the model Reference :https://github.com/keras-team/keras/issues/2110    
    # saving models at the following iterations -- uncomment it if you want to save weights and load it later
    #if iteration==1 or iteration==3 or iteration==5 or iteration==10 or iteration==20 or iteration==30 or iteration==50 or iteration==60 :
    #    model.save_weights('Karpathy_LSTM_weights_'+str(iteration)+'.h5', overwrite=True)
    #start_index = random.randint(0, len(text) - maxlen - 1)
    #sys.stdout.flush()

    print ('loss is')
    print (history.history['loss'][0])
    print (history)
    print()    

    


--------------------------------------------------
Iteration 1


/usr/local/lib/python2.7/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/5
71032/71032 [==============================] - 124s 2ms/step - loss: 2.5882 - acc: 0.3717
Epoch 2/5
31104/71032 [============>.................] - ETA: 1:08 - loss: 1.1841 - acc: 0.6955

71032/71032 [==============================] - 121s 2ms/step - loss: 0.9924 - acc: 0.7425
Epoch 3/5
50944/71032 [====================>.........] - ETA: 34s - loss: 0.5921 - acc: 0.8429

71032/71032 [==============================] - 122s 2ms/step - loss: 0.5622 - acc: 0.8507
Epoch 4/5
60032/71032 [========================>.....] - ETA: 19s - loss: 0.4167 - acc: 0.8882

...

71032/71032 [==============================] - 123s 2ms/step - loss: 0.2211 - acc: 0.9311
Epoch 2/5
53888/71032 [=====================>........] - ETA: 29s - loss: 0.2189 - acc: 0.9317

71032/71032 [==============================] - 123s 2ms/step - loss: 0.2195 - acc: 0.9316
Epoch 3/5
61312/71032 [========================>.....] - ETA: 16s - loss: 0.2179 - acc: 0.9317

71032/71032 [==============================] - 123s 2ms/step - loss: 0.2182 - acc: 0.9317
Epoch 4/5
64768/71032 [==========================>...] - ETA: 10s - loss: 0.2168 - acc: 0.9320

71032/71032 [==============================] - 123s 2ms/step - loss: 0.2169 - acc: 0.9318
Epoch 5/5
66304/71032 [===========================>..] - ETA: 8s - loss: 0.2155 - acc: 0.9323

71032/71032 [==============================] - 123s 2ms/step - loss: 0.2156 - acc: 0.9322
loss is
0.22109524240424228



#### testing
now we will use the trained model to generate code for the seed

In [0]:
seed_string="(image_memory_write)(png_structp png_ptr, png_bytep/*const*/ data," 
print ("seed string -->", seed_string)
print ('The generated text is')
sys.stdout.write(seed_string),
#x=np.zeros((1, len(seed_string), len(chars)))
for i in range(1700):
    x=np.zeros((1, len(seed_string[-40:]), len(chars)))
    for tt, char in enumerate(seed_string[-40:]):
        x[0, tt, char_indices[char]] = 1.
    preds = model.predict(x, verbose=0)[0]
    next_index=np.argmax(preds[len(seed_string[-40:])-1])
      
    next_char = indices_char[next_index]
    seed_string = seed_string + next_char
    sys.stdout.write(next_char)
sys.stdout.flush()    



seed string --> (image_memory_write)(png_structp png_ptr, png_bytep/*const*/ data,
The generated text is
(image_memory_write)(png_structp png_ptr, png_bytep/*const*/ data, .onder-bar {
  margin-top: 0;
}
.dasher-tooltip p span {
  display: block;
}
.card {
  margin-bottom: 0;
}
.one-google {
  padding-top: 27px;
}
#canvas {
  -webkit-transition: opacity 0.075s;
  -moz-transition: opacity 0.075s;
  -ms-transition: opacity 0.075s;
  transition: opacity 0.075s;
  opacity: 0.01;
}
.shift-form #canvas {
  opacity: 0.99;
}
.label {
  color: #404040;
}
#account-chooser-link {
  -webkit-transition: opacity 0.3s;
  -moz-transition: opacity 0.075s;
  -ms-transition: opacity 0.075s;
  transition: opacity 0.075s;
  opacity: 0.01;
}
.shift-form #canvas {
  opacity: 0.99;
}
.label {
  color: #404040;
}
#account-chooser-link {
  -webkit-transition: opacity 0.3s;
  -moz-transition: opacity 0.075s;
  -ms-transition: opacity 0.075s;
  transition: opacity 0.075s;
  opacity: 0.01;
}
.shift-form #canvas {


0.99;
}
.label {
  color: #404040;
}
#account-chooser-link {
  -webkit-transition: opacity 0.3s;
  -moz-transition: opacity 0.075s;
  -ms-transition: opacity 0.075s;
  transition: opacity 0.075s;
  opacity: 0.01;
}
.shift-form #canvas {
  opacity: 0.99;
}
.label {
  color: #404040;
}
#account-chooser-link {
  -webkit-transition: opacity 0.3s;
  -moz-transition: opacity 0.075s;
  -ms-transition: opacity 0.075s;
  transition: opacity 0.075s;
  opacity: 0.01;
}
.shift-form #canvas {
  opacity: 0.99;
}
.label {
  color: #404040;
}
#account-chooser-link {
  -webkit-transition: opacity 0.3s;
  -moz-transition: opacity 0.075s;
  -ms-transition: opacity 0.075s;
  transition: opacity 0.075s;
  opacity: 0.01;
}
.shift-form #canvas {
  opacity: 0.99;
}
.label {
  color: #404040;
}
#account-chooser-link {
  -webkit-transition: opacity 0.3s;
  -moz-transition: 

In [0]:
len(seed_string)

1766